# Hyperparameter optimization Notebook

## First we needed to create the data() and the create_model() funtions.

In the data function we needed to implement again the training, validation and the test data creation function. For this we also needed the VideoFrameGenerator function to be implemented in the same place.

In [ ]:
from hyperas.distributions import choice, uniform
from hyperas import optim
from hyperopt import Trials, STATUS_OK, tpe
CLASS_NUM = 10
NBFRAME = 20
#model = create_Resnet_GRU(mode='backendGRU', inputDim=64, hiddenDim=20, nClasses=CLASS_NUM, frameLen=NBFRAME, every_frame=True)

def data():
    class VideoFrameGenerator(keras.utils.Sequence):
    '''
        Generates video frames from frame directories
    '''
        def __init__(self,
                 from_dir,
                 rescale=1/255.,
                 batch_size=8,
                 shape=(100, 100, 3),
                 nbframe=20,
                 shuffle=False,
                 transform:keras.preprocessing.image.ImageDataGenerator=None,
                 grayscale=False):

            self.from_dir = from_dir
            self.rescale = rescale
            self.nbframe = nbframe
            self.batch_size = batch_size
            self.target_shape = shape
            self.shuffle = shuffle
            self.transform = transform
            self.grayscale = grayscale        
            self.classes = []
            self.files = []
            self.class_indices = {}
            self._current = 0
            self.__filecount = 0
            self.__get_all_files()

            print("\nTotal data: %d classes for %d files for %s" % (
                len(self.classes),
                len(self.files),
                os.path.basename(self.from_dir)))
    
        
        def __len__(self):
            return self.__filecount//self.batch_size

        def __iter__(self):
              return self

        def __next__(self):
            return self.next()
    
        def next(self):
            elem = self[self._current]
            self._current += 1
            if self._current == len(self):
                self._current = 0
                self.on_epoch_end()
            return elem

        def __getitem__(self, index):
            labels = []
            images = []
            # get the next block of files (it contains a batchsize of the video frames)
            frame_dirs = self.files[index*self.batch_size:(index+1)*self.batch_size]

            for dir in frame_dirs:
                # get classname from path
                label_name = self._get_classname(dir)

                # check if class name is valid
                label = np.zeros(len(self.classes))
                if label_name not in self.class_indices.keys():
                    print(f'ERROR: {label_name} not in class labels.')
                    continue
          
                # get the id of the class
                col = self.class_indices[label_name]
                label[col] = 1.

                # get preprocessed frames for the actual video
                frames = self.__readframe(dir)

                if frames is None:
                    continue

                # use data augmentation
                frames = self.__data_aug(frames)
        
                # add the sequence in batch
                images.append(frames)
                labels.append(label)

            return np.array(images), np.array(labels)
    
        def on_epoch_end(self):
            if self.shuffle:
                random.shuffle(self.files)

        def _get_classname(self, video: str) -> str:
            classname = os.path.basename(os.path.dirname(video))
            return classname
    
        def __get_all_files(self):
            # get classes and sort them in ABC order
            self.classes = glob.glob(os.path.join(self.from_dir, '*'))
            self.classes = sorted([os.path.basename(c) for c in self.classes])

            # create label indexes for classes
            self.class_indices = dict(zip(self.classes, range(len(self.classes))))

            # count all video file dirs
            self.__filecount = len(glob.glob(os.path.join(self.from_dir, '*/*')))
        
            for classname in self.classes:
                # list video file dirs for classes
                videos = glob.glob(os.path.join(self.from_dir, classname, '*'))

                self.files += videos

            # shuffle files
            random.shuffle(self.files)
            self.on_epoch_end()
        
        def __readframe(self, frame_dir):
            frames = []

            # read frame images
            files = glob.glob(os.path.join(frame_dir, '*'))
            for f in files:
                frame = cv.imread(f)
                frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
                frame = cv.resize(frame, self.target_shape[:2])
                if self.grayscale:
                      frame = cv.cvtColor(frame, cv.COLOR_RGB2GRAY) 
                frame = img_to_array(frame) * self.rescale
                frames.append(frame)
      
            if len(frames)==0:
                print("ERROR: not found frames\n")
                return None

            else:
                # use padding to create a self.nbframes length of images
                frames = pad_sequences([frames], padding="post", maxlen=self.nbframe, dtype=float, truncating="post")[0]

                # add frames in memory
                if len(frames) == self.nbframe:
                    return frames
            
                else:
                    print('\n%s has not enough frames ==> %d' % (frame_dir, len(frames)))
                    return None
            
        def __data_aug(self, frames):
            """ Make random transformation based on ImageGenerator arguments"""
            T = None
            if self.transform is not None:
                # get random transform from generator
                T = self.transform.get_random_transform(self.target_shape[:2])
        
            result = frames
            if T is not None:
                # apply transformation
                result = [self.transform.apply_transform(frame, T) for frame in frames]
    
            return np.array(result)
  
    
    NBFRAME = 20
    TARGET_SIZE = 100
    BSIZE = 32
    DATA_AUG = False
    TO_GRAYSCALE = True

    if DATA_AUG:
        data_aug = keras.preprocessing.image.ImageDataGenerator(
          horizontal_flip=True,
          rotation_range=8,
          height_shift_range=.2)
    else:
        data_aug = None

    TRAIN_DIR = '/content/visual_speech_recognition/dataset10/train'
    VAL_DIR = '/content/visual_speech_recognition/dataset10/val'
    TEST_DIR = '/content/visual_speech_recognition/dataset10/test'


    train_gen = VideoFrameGenerator(from_dir = TRAIN_DIR,
                 rescale=1/255.,
                 batch_size=BSIZE,
                 shape=(TARGET_SIZE, TARGET_SIZE, 1 if TO_GRAYSCALE else 3),
                 nbframe=NBFRAME,
                 shuffle=True,
                 grayscale = TO_GRAYSCALE,
                 transform=data_aug)


    test_gen = VideoFrameGenerator(from_dir = TEST_DIR,
                 rescale=1/255.,
                 batch_size=1,
                 shape=(TARGET_SIZE, TARGET_SIZE, 1 if TO_GRAYSCALE else 3),
                 nbframe=NBFRAME,
                 shuffle=False,
                 grayscale = TO_GRAYSCALE)
  
    val_gen = VideoFrameGenerator(from_dir = VAL_DIR,
                 rescale=1/255.,
                 batch_size=1,
                 shape=(TARGET_SIZE, TARGET_SIZE, 1 if TO_GRAYSCALE else 3),
                 nbframe=NBFRAME,
                 shuffle=False,
                 grayscale = TO_GRAYSCALE,
                 transform=data_aug)

    return train_gen,val_gen, test_gen


In the create_model() function we implemented again the training function, but now we used hyper parameter.
We declared 5 different hyperparameter, those are as follows: activation, dropout, optimizer, hiddenDim, inputDim.

In the activation parameter, the function can choose among: relu, elu and tanh.
The dropout should be between 0 and 0,4.
The optimizer can be selected among, rmsprop, adam, and sgd.
For the hidden dimension and the input dimension we gave discret values, because those can't be float numbers.

We also save the best data for each evaluation into a csv file. The header of this file is mostly the above mentioned hyper parameters, and the validation accuracy.

In [ ]:
def create_model(train_gen, val_gen):

    CLASS_NUM = 10

    import tensorflow as tf
    from tensorflow.keras.layers import Input, Dense, GRU, GlobalAveragePooling2D, MaxPooling1D, Lambda, Bidirectional
    from tensorflow.keras.layers import MaxPooling3D, ZeroPadding3D, Conv3D, Conv1D, Activation, BatchNormalization, Dropout
    from keras.optimizers import Adam, SGD, RMSprop
    from tensorflow.keras import Model, Sequential
    from classification_models.keras import Classifiers
    from hyperas.distributions import choice, uniform
    from hyperas import optim

    # hyper parameter ranges
    activation = {{choice(['relu', 'elu','tanh'])}}
    dropout = {{uniform(0, 0.4)}}
    optimizer = {{choice(['rmsprop', 'adam', 'sgd'])}}
    hiddenDim = {{choice([20,50,80])}}
    inputDim = {{choice([32,64,128])}}

    def GRU(x, hidden_size):
        # in case of GRU add reset_after=False parameter
        out = Bidirectional(keras.layers.GRU(hidden_size, return_sequences=True, kernel_initializer='Orthogonal', reset_after=False, name='gru1'), merge_mode='concat')(x)
        out = Bidirectional(keras.layers.GRU(hidden_size, return_sequences=False, kernel_initializer='Orthogonal', reset_after=False, name='gru2'), merge_mode='concat')(out)
        return out

    def create_Resnet_GRU(mode,  nClasses, frameLen,activation, dropout, optimizer,input_shape=(NBFRAME,TARGET_SIZE,TARGET_SIZE,1),
                        every_frame=True, hiddenDim=hiddenDim, inputDim = inputDim):
        frontend3D = Sequential([
                ZeroPadding3D(padding=(2, 3, 3)),
                Conv3D(32, kernel_size=(5, 7, 7), strides=(1, 2, 2), padding='valid', use_bias=False),
                BatchNormalization(),
                Activation(activation),
                ZeroPadding3D(padding=((0, 4, 8))),
                MaxPooling3D(pool_size=(1, 2, 3), strides=(1, 1, 2))
                ])
  
        # for temporal convolution
        backend_conv1 = Sequential([
                Conv1D(2*inputDim, 5, strides=2, use_bias=False),
                BatchNormalization(),
                Activation(activation),
                MaxPooling1D(2, 2),
                Conv1D(4*inputDim, 5, strides=2, use_bias=False),
                BatchNormalization(),
                Activation(activation),
                ])
  
        # for temporal convolution
        backend_conv2 = Sequential([
                Dense(inputDim),
                BatchNormalization(),
                Activation(activation),
                Dense(nClasses)
                ])
  
        # -----------------------------------------------------------------
        # creating the model
        input_frames = Input(shape=input_shape, name='frames_input')
        x = frontend3D(input_frames)

        # reshape output for the input of Resnet 2D
        x = Lambda(lambda x : tf.reshape(x, [-1, int(x.shape[2]), int(x.shape[3]), int(x.shape[4])]), name='lambda2')(x)

        channels = int(x.shape[-1])
  
        # get resnet model
        ResNet18, preprocess_input = Classifiers.get('resnet18')
        resnet18 = ResNet18((None, None, channels), weights=None, include_top=False)

        x = resnet18(x)

        # Flatten with global average pooling  for the input of the dense layer
        x = GlobalAveragePooling2D(name='global_avgpool_resnet')(x)
        x = Dense(inputDim, name='dense_resnet')(x)
        x = Dropout(dropout)(x)
        x = BatchNormalization(name='bn_resnet')(x)

        if mode == 'temporalConv':
                x = Lambda(lambda x : tf.reshape(x, [-1, frameLen, inputDim]), name='lambda3')(x)   #x.view(-1, frameLen, inputDim)
                x = Lambda(lambda x : tf.transpose(x, [0, 2, 1]), name='lambda4')(x)   #x.transpose(1, 2)
                x = backend_conv1(x)
                x = Lambda(lambda x : tf.reduce_mean(x, 2), name='lambda5')(x)
                x = backend_conv2(x)

        elif mode == 'backendGRU' or mode == 'finetuneGRU':
                x = Lambda(lambda x : tf.reshape(x, [-1, frameLen, inputDim]), name='lambda6')(x)    #x.view(-1, frameLen, inputDim)
                # add memory cells to the network (GRU or LSTM)
                x = GRU(x, hiddenDim)

                # add the dropout layer and the last Dense layer for classification
                if every_frame:
                    x = Dropout(dropout)(x)
                    x = Dense(nClasses, activation='softmax')(x)  # predictions based on every time step
                else:
                    x = Dropout(dropout)(x)
                    x = Dense(nClasses, activation='softmax')(x[:, -1, :])  # predictions based on last time-step

                model = Model(inputs=input_frames, outputs=x)

        return model

    # construct the model
    model = create_Resnet_GRU(mode='backendGRU', inputDim=inputDim,hiddenDim = hiddenDim,  nClasses=CLASS_NUM, frameLen=NBFRAME, every_frame=True,
                            activation = activation, dropout = dropout, optimizer = optimizer)
    # choose the optimizer according to the given string
    if optimizer == "rmsprop":
        optim = keras.optimizers.RMSprop(learning_rate = 0.001, momentum = 0.9)
    elif optimizer == "adam":
        optim = keras.optimizers.Adam(learning_rate = 0.001)
    else:
        optim = keras.optimizers.SGD(learning_rate = 0.001, momentum = 0.9)

    model.compile(optimizer=optim,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
    callbacks = [keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, verbose=0)]
  
    result = model.fit_generator(train_gen,
                epochs=40,
                verbose=2,                
                validation_data=val_gen,
                callbacks = callbacks,
                shuffle=True)
  
    acc = np.amax(result.history['val_accuracy']) 

    #dropout, activation, optimizer, n_batch
    with open('/content/drive/MyDrive/hyperas-log.csv', 'a') as csv_file:
        csv_file.write(str(dropout) + ';')
        csv_file.write(str(activation) + ';')
        csv_file.write(str(optimizer) + ';')
        csv_file.write(str(hiddenDim)+';')
        csv_file.write(str(inputDim)+';')
        csv_file.write(str(64) + ';')   
        csv_file.write(str(acc) + '\n')

    return {'loss': -acc, 'status': STATUS_OK, 'model':model}

## Initializing the CSV file header

We also added the batch size, but we mostly used 32, because it was the one which gave the best results.

In [ ]:
# initialize log file
with open('/content/drive/MyDrive/hyperas-log.csv', 'w') as csv_file:
    csv_file.write('dropout' + ';')
    csv_file.write('activation' + ';')
    csv_file.write('optimizer' + ';')
    csv_file.write('hiddenDim' + ';')
    csv_file.write('inputDim' + ';')
    csv_file.write('n_batch' + ';') 
    csv_file.write('acc' + '\n')

## Importing the moduls for the optimization

In [ ]:
import hyperas
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

## Optimization 

In [ ]:
best_run, best_model = optim.minimize(
     model=create_model,
     data=data,
     algo=tpe.suggest,
     max_evals=100,
     notebook_name='hyperparameter_opt',
     trials= Trials())

## Visualization of the result 

In [1]:
import pandas
log_file = pandas.read_csv('/content/drive/MyDrive/hyperas-log.csv', delimiter=';')

best10 = log_file.sort_values(by=['acc'], ascending=False).head(n=10)
best10

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/hyperas-log.csv'

### Drawing heatmaps, one for the hidden dimensions, and one representing the input dimension changes 

In [ ]:
import seaborn as sns

max_val_acc = log_file.groupby(['hiddenDim', 'optimizer']).max()
max_val_acc = max_val_acc.unstack()[['acc']]
sns.heatmap(max_val_acc.acc, annot=True, fmt='.4g');

b, t = plt.ylim()
b += 0.5
t -= 0.5
plt.ylim(b, t)

In [ ]:
max_val_acc = log_file.groupby(['inputDim', 'optimizer']).max()
max_val_acc = max_val_acc.unstack()[['acc']]
sns.heatmap(max_val_acc.acc, annot=True, fmt='.4g');

b, t = plt.ylim()
b += 0.5
t -= 0.5
plt.ylim(b, t)